# Application 1: Visualizing a particular setup

On FUMBBL there is a great guide on defensive setups. Suppose we wish to visualize the "arrowhead" setup, advised for Undead.

_The arrowhead defense is a good defense for Undead, Necromantic, or Chaos Renegades, against a highly mobile opponent. It’s similar to a ziggurat or chevron defense, but the 3-column midfielders or safeties have been moved into the 0-column to prevent runs up the gut, and also to be able to redeploy from a central position, while the 2-column strong midfielders have been pushed out to the 3-column to form a spine screen. It’s strong in the wide zone and up the center, but the 3-column midfielders are seriously exposed, and unless they have both a lot of Strength and the Stand Firm skill, this position is highly vulnerable._

The Arrowhead Defense

```{text}
7 6 5 4|3 2 1 0 1 2 3|4 5 6 7   column
- - - -|- - x x x - -|- - - -   Line
- - - -|- - - - - - -|- - - -   -1
- - x -|x - - x - - x|- x - -   -2
- x - -|- - - x - - -|- - x -   -3
```

In [ ]:
fb.show_boardpos(rotation = 'H')

In [ ]:
roster = fb.fetch_roster("Shambling Undead")
roster

In [ ]:
my_setup = ['setup', ['Z1: g14', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16', 'G1: h16', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]


positions = fb.create_position(roster, my_setup)

fb.create_plot(positions, receiving_team = "teamAway", orientation = 'V', crop = "lower")

for whats the play we need to display prone and stunned as well. Should include this in the notation. Maybe x and /

my_setup = ['setup', ['Z1: g14x', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16/', 'G1: h16/', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]

# Application 2: saving defensive setups as PNGs for use in playbooks

We want to plot all defensive setups together with match outcome from the Tilean Team Cup.
The Tilean Team Cup was an online NAF tournament held on FUMBBL from march 2023 to may 2023.

In [ ]:
# point this to the location of the HDF5 datasets
path_to_datasets = '../fumbbl_datasets/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')


tilean_replays = df_matches['match_id'].values

tilean_replays = tilean_replays[0:3]
tilean_replays

In [ ]:
fullrun = 0

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id)  # gnome 4543329 #4528210 #4542768
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = True)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[2])
        race_offense.append(metadata[3])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
df_replays

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/wood_elf/1606445_4447434_kickoff_lower_defense.png")